In [ ]:
!-H pip3 install --upgrade pip
!-H pip2 install --upgrade pip

In [ ]:
# Stable Baselines only supports tensorflow 1.x for now
#%tensorflow_version 1.x
import sys 
!{sys.executable} -m pip install tensorflow==1.15.0

!pip install stable-baselines[mpi]==2.10.0

In [ ]:
import numpy as np

import gym
from gym import Env
from gym.spaces import Box
from gym.utils import seeding

import math

model_name12 = "ppo2_012_exponential_reward"
model_name16 = "model_016"
model_name17 = "model_017"

class DifferentialDriveEnv(Env):
  """Custom Environment that follows gym interface"""
  # metadata = {'render.modes': ['human']}
  metadata = {'render.modes': ['console']}

  def __init__(self, L, r, delta_t = 0.01,
               init_position=None, goal_position=[0,0],
               goal_threshold = 0.1, max_duration = 500,
               min_action=-1, max_action=1,
               min_position=[-1,-1], max_position=[1,1]):
    
    super(DifferentialDriveEnv, self).__init__()

    #Define model parameters
    self.L = L 
    self.r = r

    self.orientation_map = math.pi
    self.action_map = 3

    self.delta_t = delta_t
    self.threshold = goal_threshold

    # Define action and observation space
    # They must be gym.spaces objects
    self.min_action = min_action
    self.max_action = max_action
    self.min_position = min_position
    self.max_position = max_position

    self.min_orientation = -1
    self.max_orientation = 1
    
    self.init_position = init_position
    self.goal_position = goal_position

    # self.goal_velocity = goal_velocity
    # self.goal_orientation = goal_orientation
    self.goal_reached_count = 0

    self.max_duration = max_duration
    self.duration = self.max_duration
    #self.max_duration = 250
    
    self.low_state = np.array(
        self.min_position+[self.min_orientation], dtype=np.float32
    )

    self.high_state = np.array(
        self.max_position+[self.max_orientation], dtype=np.float32
    )

    self.viewer = None

    self.action_space = Box(
        low=self.min_action,
        high=self.max_action,
        shape=(2,),
        dtype=np.float32
    )

    self.observation_space = Box(
        low=self.low_state,
        high=self.high_state,
        shape=(3,),
        dtype=np.float32
    )

    self.seed()
    self.reset()

  def seed(self, seed=None):
    self.np_random, seed = seeding.np_random(seed)
    return [seed]

  def reset(self):
    # Reset the state of the environment to an initial state

    if self.init_position is None:
      self.state = np.array([self.np_random.uniform(low=-0.1, high=0.1), self.np_random.uniform(low=-0.5, high=0.5), -1/2 + self.np_random.uniform(low=-0.01, high=0.01)])      
      #self.state = np.array([self.np_random.uniform(low=-0.03, high=-0.02), self.np_random.uniform(low=-0.02, high=-0.01), -math.pi/2])
      #self.state = np.array([self.np_random.uniform(low=0.02, high=0.03), self.np_random.uniform(low=-0.02, high=-0.01), -math.pi/2])
      # self.state = np.array([self.np_random.uniform(low=self.min_position[0], high=self.max_position[0]), self.np_random.uniform(low=self.min_position[0], high=self.max_position[0]), math.pi/2])
    elif isinstance(self.init_position, list):
      if len(self.init_position) == 3:
        self.state = np.array(self.init_position)
      else:
        raise Exception("Initial position must be size 3: [x, y, theta]")
    else:
      raise Exception("Initial position must be a list: [x, y, theta]")
    #self.max_duration = 500
    self.duration = self.max_duration
    return np.array(self.state)

  def render(self, mode='console', close=False):
    if mode is 'console':
      print("========================================================")
      print(">> Pos: x = ",self.state[0],"; y = ",self.state[1])
      print(">> Ori: ",self.state[2])
      print("========================================================")

  def step(self, action):
    x = self.state[0]
    y = self.state[1]
    theta = self.state[2]*self.orientation_map

    v = self.action_map*(action[1] + action[0]) * self.r / 2      #max vlin = 2 * 3 *0.15 /2 = 3*0.15 = 0.45 m/s (was 0.75 wmax = 5)
    w = self.action_map*(action[1] - action[0]) * self.r / self.L #max omega = 3 - (-3) * 0.15 /0.5 = 0.9 /0.5  1,8 rad/s ()

    x = x + v * self.delta_t * math.cos(theta)
    y = y + v * self.delta_t * math.sin(theta)
    theta = theta + w * self.delta_t

    if theta > math.pi:
        theta = theta - 2*math.pi
    elif theta < -math.pi:
        theta = (2*math.pi + theta)
    
    #threshold = 0.1
    #threshold = 0.001
    distance_to_target = np.linalg.norm(np.array(self.goal_position)-np.array([x, y]))

    goal_reached = bool(distance_to_target <= self.threshold)
    time_expired = bool(self.duration <= 0)
    too_far = bool(distance_to_target > 0.1)

    reward = 0
    #beta = 10
    #sigma = 0.3

    #if done:
    if goal_reached:
      reward += 10000.0
      self.goal_reached_count += 1
    else:
      #reward -= distance_to_target*0.1
      reward -= distance_to_target**2 + (np.arctan2(y,x) + np.pi - theta)**2
      #reward -= (np.arctan2(y,x) + np.pi - theta)**2
      #reward = beta*np.exp(-(distance_to_target**2)/(2*sigma**2))
      if time_expired:
        reward -= 500

    done = goal_reached or time_expired

    info = {}

    self.state = np.array([x, y, theta/self.orientation_map])

    self.duration -= 1

    return self.state, reward, done, info

  def close(self):
    pass

In [ ]:
from stable_baselines import DQN, PPO2, A2C, ACKTR, TRPO
from stable_baselines.common.cmd_util import make_vec_env
from stable_baselines.common.policies import MlpPolicy


# Instantiate the env
#env = DifferentialDriveEnv(L=50, r=15)

init_pose = [0.3, 0.3, np.pi]
env = DifferentialDriveEnv(init_position=init_pose, goal_threshold = 0.1, L=0.5, r=0.12, max_duration = 1000)
#env = DifferentialDriveEnv(init_position=init_pose,goal_threshold = 0.05,L=0.5, r=0.16)
#env = DifferentialDriveEnv(init_position=init_pose,goal_threshold = 0.05,L=0.5, r=0.17)

# wrap it
env = make_vec_env(lambda: env, n_envs=4)

# Keep on training an pre-existing model
#model = PPO2.load(model_name12)

# Train the agent
model = PPO2(MlpPolicy, env, verbose=1)
#model.set_env(env)
model.learn(total_timesteps=50000000)
model.save(model_name12) 

In [ ]:
model.save(model_name12) 

In [ ]:
print("Goal reached in env 0: {} times".format(env.envs[0].goal_reached_count))
print("Goal reached in env 1: {} times".format(env.envs[1].goal_reached_count))
print("Goal reached in env 2: {} times".format(env.envs[2].goal_reached_count))
print("Goal reached in env 3: {} times".format(env.envs[3].goal_reached_count))

del env
del model

In [ ]:
#env = DifferentialDriveEnv(L=50, r=15)
init_pose = [0.3, 0.3, np.pi]
env = DifferentialDriveEnv(L=0.5, r=0.12, init_position=init_pose)
model = PPO2.load(model_name12) #typo it should be ppo_etc
obs = env.reset()
while False:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render(mode = 'console')

In [ ]:
def show_rl_trajectory(obs_list):
    x_values = list(map(lambda obs: obs[0], obs_list))
    y_values = list(map(lambda obs: obs[1], obs_list))
    theta_values = list(map(lambda obs: obs[2], obs_list))
    print("Starting point: x:{}, y:{} -PURPLE-".format(x_values[0],y_values[0]))
    print("End point: x:{}, y:{} -GREEN-".format(x_values[-1],y_values[-1]))
    def on_close(event):
        print('Closed Figure!')
        
    fig = plt.figure()
    fig.canvas.mpl_connect('close_event', on_close)
    plt.scatter(x_values, y_values,color='blue')
    plt.scatter(x_values[0],y_values[0],color='purple')
    plt.scatter(x_values[-1],y_values[-1],color='green')
    plt.scatter(0,0,color='red',marker='x')
    plt.axis("equal")
    plt.grid()
    plt.show()

In [ ]:
import matplotlib.pyplot as plt

obs = env.reset()
n_steps = 5000
score = 0.0
history = []
obss = []
obss.append(obs)
history.append(score)
for step in range(n_steps):
  action, _ = model.predict(obs, deterministic=True)
  print("Step {}".format(step + 1))
  print("Action: ", action)
  obs, reward, done, info = env.step(action)
  print('obs=', obs, 'reward=', reward, 'done=', done)
  env.render(mode='console',close = True)
  score+=float(reward)
  history.append(score)
  obss.append(obs)
  if done:
    # Note that the VecEnv resets automatically
    # when a done signal is encountered
    print("Goal reached!", "reward=", reward)
    break

plt.plot(history)
plt.title('Episode score')
plt.ylabel('Score')
plt.xlabel('Steps')
plt.show()
show_rl_trajectory(obss)

In [ ]:
del env
del model